# Hey You

# Yes You

# Read the Comments and Directions !!!!

# I mean, ... they are directly inline in the document

# It is very easy to do

# Just read them before coming to me ...

## also ... Google is your friend

# Imports

If you get an error of the form: <br><br>`ModuleNotFoundError: No module named ...` <br><br> then you need to install that package, and possibly update it as well.
The script imports.sh should help install and update everything for you, but if there was something missed and you need to install it, first try: <br><br> `conda install name-of package` <br><br> If that doesn't work, try <br><br> `./[path to your anaconda3 directory]/bin/pip install name-of-package` <br><br> (obviously substituting the path for your anaconda directory).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import skimage as sk
from skimage import measure
from timeit import default_timer as timer
from scipy.spatial import ConvexHull
from functools import reduce

from enum import Enum
import pandas as pd
import os.path
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import dask.dataframe as dd
import resource
import holoviews as hv
import datashader as ds
import time
from bokeh.models import HoverTool

hv.notebook_extension()
hv.extension('bokeh')
from holoviews.operation.datashader import aggregate, datashade, dynspread, shade

from datashader import transfer_functions as tf
from colorcet import fire, gray, b_cyclic_wrwbw_40_90_c42
from holoviews.operation.datashader import datashade

from mothermachine.abstract_mother_machine_file import AbstractMotherMachineFile
from mothermachine.plot_conn_comp_settings import PlotConnCompSettings
from mothermachine.plot_conn_comp_results import PlotConnCompResults
from mothermachine.tools import make_cell_montage
from mothermachine.mm_file_io import save_properties, load_properties, detect_image_directories, find_shared_lane_pos_time_indcs

from mothermachine.segmentation.extract import extract_connected_components_standard, extract_cells
from mothermachine.segmentation.cleanup import select_cells_in_trenches, select_reasonable_cells
from mothermachine.segmentation.lineages import sort_cells_into_lineages, determine_position_of_cell_in_trench, create_linear_index
from mothermachine.segmentation.trenchlocs import TrenchLocs
from mothermachine.post_cluster_analysis import stack_mother_properties, unstack_mother_properties, add_mother_property, find_peaks
from mothermachine.analyze_division import  calculate_and_append_divisions, cleanup_lengths

%matplotlib inline
plt.rcParams['figure.figsize'] = (18.0, 8.0)


# Define Segementation File Class

The Abstract Class AbstractMotherMachineFile must be defined to allow file-names to be generated on the fly when supplied with a lane_num, pos_num and t_frame. You will need to define it for at least the Segmentation file but also for any fluorescent images that aren't used for segmentation. See examples below:

In [ ]:
# you may need to change this depending on your operating-system type
example_base_dir = "/run/user/1001/gvfs/smb-share:server=research.files.med.harvard.edu,share=sysbio/PAULSSON LAB/MotherMachinePython/sample_images/"

## Load example files

1. I want to run [Example 1](#Example-1).
2. No, I want to run [Example 2](#Example-2).
3. No, I want to analyze my old data [Load Old Data](#load-old-data)

## Example 1

In [ ]:
basedir = os.path.join(example_base_dir,'example1')

def generate_save_directory(lane_num):
    save_dir = os.path.join(basedir, "Lane{0:0>1}".format(lane_num))
    return save_dir

class MotherMachineFile(AbstractMotherMachineFile):
    n_img_channels = 2
    
    def set_base_directory(self):
        return basedir
    
    def construct_image_directory(self):
        lane_dir = "Lane{lane_num:0>1}".format(lane_num = self.lane_num)
        pos_dir = "pos{pos_num:0>1}".format(pos_num = self.pos_num)
        return os.path.join(self.basedir, lane_dir, pos_dir)

    def construct_filename(self):
        # currently using a file_num as a hack for including fluor files
        if self.img_channel == 0:
            return "sb27_pos{pos_num:0>1}-r_raw-{t_frame:0>3}.tif".format(pos_num = self.pos_num, t_frame = self.t_frame)
        elif self.img_channel == 1:
            return "sb27_pos{pos_num:0>1}-g_raw-{t_frame:0>3}.tif".format(pos_num = self.pos_num, t_frame = self.t_frame)

Awesome, I am done and am ready to verify my MotherMachineFile class is defined correctly [Check MotherMachineFile](#Check-MotherMachineFile)

## Example 2

You may notice that the stanadard segmentation isn't working that well when the cells enter stationary phase in this example, around timepoints > 80. Consider adjusting the default settings for the extract_connected_components_standard.
hint: it is combinding lineages together, so consider reducing the maxima smoothing parameter ... you can also try to play with other maxima parameters to get more agressive watershed cutting, it also clogs, so to do it properly you need to crop the image in your extract_conn_comps_func, consider definining a simple custom function using the standard function as the basis, but that also includes a cropping step

In [ ]:
basedir = os.path.join(example_base_dir,'example2')

def generate_save_directory(lane_num):
    save_dir = os.path.join(basedir, "Lane_{lane_num:0>2}".format(lane_num = lane_num))
    return save_dir

class MotherMachineFile(AbstractMotherMachineFile):
    n_img_channels = 1
    
    def set_base_directory(self):
        return basedir
    
    def construct_image_directory(self):
        lane_dir = "Lane_{lane_num:0>2}".format(lane_num = self.lane_num)
        pos_dir = "pos_{pos_num:0>3}".format(pos_num = self.pos_num)
        return os.path.join(self.basedir, lane_dir, pos_dir)

    def construct_filename(self):
        # currently using a file_num as a hack for including fluor files
        if self.img_channel == 0:
            return "SB7--GC_EXIT_4_MEDIA_001_pos_{pos_num:0>3}_t{t_frame:0>4}_c_MCHERRY.tiff".format(pos_num = self.pos_num,t_frame = self.t_frame)
  

Awesome, I am done and am ready to verify my MotherMachineFile class is defined correctly [Check MotherMachineFile](#Check-MotherMachineFile)

## Check MotherMachineFile

Run the cell below, if you see the correct filename followed by a matrix, your class is probably reasonable, if you get a bunch of errors, you probably defined your class wrong or you chose a lane, pos, t_frame or img_channel that doesn't exist

In [ ]:
# set these values to ones you know exist
lane_num = 1
pos_num = 2
t_frame = 1
img_channel = 0
s_file = MotherMachineFile(lane_num,pos_num,t_frame)
print("Is this your filename: \n\n" + s_file.fullfile)
print("\n Do you see a reasonable matrix below? \n")
s_file.getImage()

I am a champ and everything looks sweet. I am ready to specify the lanes, positions and time frames for analysis [specify](#Specify-lanes,-positions-and-time-frames)

# Specify lanes, positions and time frames

In [ ]:
# these are somewhat experimental ways to find all the lanes, positions and time_points for your data using the mother machine file you define
# this should also tell you whether you messed up when specifying your filenames, if this doesn't show anything below, you probably messed up
lane_pos_time_list = detect_image_directories(MotherMachineFile)
lane_indcs, pos_indcs, tindcs = find_shared_lane_pos_time_indcs(lane_pos_time_list)
lane_indcs, pos_indcs, tindcs

In [ ]:
# if the above didn't work but you are sure everything is normal you can specify them manually below
#lane_indcs= [...], pos_indcs = [...], tindcs=[...]

* Everything is looking good. I am ready to startup a parallel environment with dask. [dask](#Start-Dask-Client)
* I am a really inquisitive person, tell me about the standard connected components extraction function [details](#Definition-of-standard-conn-comp-extraction-function)
* Actually, I am an expert and want to specify a cusom connected component extraction function [define](#define-any-custom-conn-comp-extraction-function)

# If analyzing already extracted data

## load old data

In [ ]:
# load old parameters (if not calculating new properties)
##### only if using previously calculated and saved properties #####
# you should define this generate_load_directory and basedir below

# basedir='...'
def generate_load_directory(lane_num):
    load_dir = os.path.join(basedir, "Lane{0:0>1}".format(lane_num))
    return load_dir

In [ ]:
# load old parameters (if not calculating new properties)
##### only if using previously calculated and saved properties #####3

lanes_to_load = [1]
props_all = pd.DataFrame()

for lane in lanes_to_load:
    root_dir = generate_load_directory(lane)
    props_all = props_all.append(load_properties(os.path.join(root_dir, 'props_all.pkl')),ignore_index=True)

# Connected Component Extraction Functions

## Definition of standard conn comp extraction function


The standard extraction function is called extract_connected_components_standard. It involves finding a rough segmentation using a global otsu threshold and substantial dilation. By multiplying the subsequent segmentation by this rough_thresh, background junk is removed. The main segementation is an auto-local-threshold on a gaussian smoothed image using the niblack method. Next, the centroids ('maxima' in the function) of each cells are found using an auto-local-threshold on a gaussian smoothed image using a niblack method, but with a much smaller niblack window. These are fed into a special watershed algorithm that spits out connected components. This segementation can be tuned by specifying the named parameters, for example by using a higher init_smooth_sigma for noisier images. Alternatively, a completely different segmentation algorithm can be devised. For the pipeline, you just need to provide a connected components matrix to the extract_cells function (as well as the MotherMachineFile used for segmentation and any additional fluorescent MotherMachinefiles.

## define any custom conn comp extraction function

This is where you will specify any non-standard function for extracting components
Normally you will not need to do this.

In [ ]:
# extract_connected_components_func = ...

# Start Dask Client

Dask is library that sets up the parallelization. You start a client that runs several workers, called a 'cluster'.
If running locally, set run_local=True. 
Also, you can specify the initial number of workers to spawn. 
If running locally the max number of workers is the number of cpu cores. 
If running on cluster, the n_workers_init is just the initial number of workers to spawn (keep this small!!!!). You are going to want to test all your settings and run a small extraction using a small subset of your data before scaling to a large number of workers. You can scale up or down as will be shown below.
You can also see the active processing by clicking the link next to Dashboard (assuming you are on the correct network)


In [ ]:
run_local = True
n_workers_init = 6

if run_local:
    client = Client()
    client.cluster.scale(n_workers_init)
    time.sleep(0.5) # allow client time to respond to scaling before display
else:
    # note the specifed walltime, don't use too much or too little, 01:30:00 is a good baseline, 
    # you just need enough time to finish 'gathering' to props_all before the jobs die
    # you can always spin up more jobs later
    # you will launch many jobs, so you don't need multiple processes, a lot of ram or multiple threads
    cluster = SLURMCluster(queue="short",walltime='01:30:00',job_cpu=1,job_mem='6G',threads=1,processes=1,memory='6GB')
    cluster.start_workers(n_workers_init)
    client = Client(cluster)

client

If everything looks good, proceed through each of the sections below. Good luck!

# Determine optimal connected component extraction settings

This class is designed to provide a way to play with the parameters of a standard connected components extraction procedure, i.e. one that uses the parameters (init_niblack_k, maxima_niblack_k, init_smooth_sigma, maxima_smooth_sigma, init_niblack_window_size, maxima_niblack_window_size). <br> <br>
Note: the plot may take up to a minute to initially load and may spit out a warning before appearing (ignore this, there is a bug in the holoviews code). <br> <br> Also Note: the colors do not indicate lineages, i.e. it is expected to change as you go through t-frames, the colors are just used to distinguish connected components within a single image. Also, identical colors do not necessarily indicate they are the same region.<br> <br>
Finally: it is supposed to start out zoomed in on part of the image, use the tools to zoom or move around.

In [ ]:
# specify a subset of all tindcs for speed
tindcs_to_plot = np.sort(np.random.choice(tindcs,size=15,replace=False))
conn_comp_plotter = PlotConnCompSettings(lane_indcs, pos_indcs, tindcs_to_plot, MotherMachineFile,
                                         extract_connected_components_standard, client, has_two_lanes = True)
conn_comp_plotter.plot()

# Extract Connected Components


This defines the run_extraction function for doing parallel extraction of a list of dictionary containing region props for all connected components found with the connected_components_func. All that is needed is MotherMachineFile object, corresponding connected component (or procedure for generating one). <br>

use_parameters_from_plot = [True,False] : if True, then the optional parameters specified by the conn_comp_plotter settings shown in the plots above will be used, otherwise it will use defaults. <br> <br>
props_to_grab = ['all','min','supp'] : (advanced-users only) if you specify min, it will only gather a smaller set of parameters, this can be a lot faster but you won't get all the useful parameters at the end, i.e. solidity, convex hull, etc. Only really helpful it you have a very large number of regions you won't need to filter with advanced properties later on.

## Specify Extraction Function

In [ ]:
use_parameters_from_plot = True
props_to_grab = 'all' #choose ['all','min','supp] (default: 'all', only use if you know what you are doing)

if use_parameters_from_plot:
    kwargs = (conn_comp_plotter.recC.kwargs.copy())
    print("Using these parameters for analysis: \n")
    print(kwargs)
    extract_conn_comp = conn_comp_plotter.extract_conn_comp_func
    def extract_connected_components_func(img):
        return extract_conn_comp(img, **kwargs)

n_img_channels = MotherMachineFile.n_img_channels

def run_extraction(lane_num, pos_num, t_frame):
    s_file = MotherMachineFile(lane_num,pos_num,t_frame)
    f_files = [MotherMachineFile(lane_num, pos_num, t_frame, i) for i in range(1, n_img_channels)]
    conn_comp = extract_connected_components_func(s_file.getImage())
    return extract_cells(s_file, conn_comp, fluorescent_files = f_files, props_to_grab = props_to_grab)

## Run Extraction

The parameters lane_num_to_analyze, pos_indcs_to_analyze, t_indcs_to_analyze will specify which lanes, positions, and time frames you want to analyze. If you are running on the cluster, the parameter scale_workers will specify the number of jobs to launch total.

In [ ]:
#### specify me !!!!!!! #####
lane_num_to_analyze = lane_indcs[0]
pos_indcs_to_analyze = pos_indcs[0:2]
t_indcs_to_analyze = tindcs[0:30]
scale_workers = 0 # set to zero if running locally
#### end specify me !!!!!!! #####

if scale_workers > 0:
    cluster.scale(scale_workers)
    
futures = []
for pos_num in pos_indcs_to_analyze:
    
    def extract(t_frame):
        return run_extraction(lane_num_to_analyze,pos_num,t_frame)
    
    fut = client.map(extract,t_indcs_to_analyze)
    futures.append(fut)

all_futures = [f for sublist in futures for f in sublist]

dask.distributed.progress(all_futures)

## put properties into pandas dataframe and save

In [ ]:
## combine all results together ##
# by combining in a loop, ram usage is minimized

props = client.gather(futures[0])
flat_props = [item for sublist in props for item in sublist]
props_all = pd.DataFrame(flat_props)
int_dtype_list = props_all.select_dtypes(include=np.int64).dtypes.index.tolist()
props_all[int_dtype_list] = props_all[int_dtype_list].astype(np.uint16)

for i in range(1,len(futures)):
    props = client.gather(futures[i])
    flat_props = [item for sublist in props for item in sublist]
    add_props = pd.DataFrame(flat_props)
    add_props[int_dtype_list] = add_props[int_dtype_list].astype(np.uint16)
    props_all = props_all.append(add_props,ignore_index=True)
    
print("mem used: %2.3f" % (int(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)/(10**6)))

You should now have your result! Awesome! <br>
It is in a pandas dataframe called props_all. I have included some basic commands to verify that the extraction worked. <br>
You should also checkout the pandas docs to get a sense of the cool things you can do with that dataframe.

In [ ]:
# some helpful funtions to verify that the extraction worked correctly
# uncomment one at a time to view them

### show first few lines
props_all.head()

### show descriptive statistics for all of the columns
#props_all.describe()

I highly recommend saving your result. <br>
Verify the save directory is correct below, then save.

In [ ]:
save_dir = generate_save_directory(lane_num)
save_dir

In [ ]:
save_properties(props_all, os.path.join(save_dir, "props_all.pkl"))

# Analyze and filter data

In [ ]:
property_bounds_dict = {}
property_bounds_dict['area'] = [14,10000]
# property_bounds_dict['solidity'] = [0.8,  1.0] # just an example of the types of bounds that can be specified

start = timer()
props_clean = select_cells_in_trenches(props_all,trenchLocs = TrenchLocs.TOP_AND_BOTTOM, below_trench_quantile=70,above_trench_quantile=90,mother_cell_y_offset=15)
props_clean = select_reasonable_cells(props_clean,property_bounds_dict)
end = timer()

print("time elapsed: %2.3f" % (end-start))

## show all the centroids from all the positions z-stacked on top of eachother

This visualization can be used to verify that the correct filtering was applied above, i.e. if the dots are too tight in the y-direction, relax the quantiles, if they are too loose tigthen them, etc. Go back to [filter](#Analyze-and-filter-data) and tweak until you are happy with the result. <br><br>
When you are happy with the extent of centroids continue on to plot them overlayed on the images as a final sanity check. <br><br>
You can either plot the convex hull, which is very fast and should give you essentially the information you need, or you can scatter all the centroids which is slow and ram intensive but gives more detail. Unless something is broken, use quick_plot=True

In [ ]:
img_width, img_height = props_clean.iloc[0].img_width, props_clean.iloc[0].img_height
img_width_scaled = int(0.65*img_width)
img_height_scaled = int(0.65*img_height)

In [ ]:
%%opts Scatter [title_format='z-stacked centroids' width=img_width_scaled height=img_height_scaled invert_yaxis=True fontsize={'title':30, 'xlabel':10, 'ylabel':10, 'ticks':10}] (size=10)
%%opts Curve [title_format='z-stacked centroids convex hull' width=img_width_scaled height=img_height_scaled invert_yaxis=True fontsize={'title':30, 'xlabel':10, 'ylabel':10, 'ticks':10}]

quick_plot = True

if quick_plot:
    def plot_hulls(pos_num):
        tims = np.unique(props_clean.trench_inversion_mult.values)
        p=[]
        for tim in tims:
            to_select = np.all((props_clean.trench_inversion_mult == tim, props_clean.pos_num == pos_num),axis=0)
            cents = props_clean.loc[to_select,['centx','centy']].values
            hull = ConvexHull(cents)
            verts = np.append(hull.vertices,hull.vertices[0])
            p.append(hv.Curve((cents[verts,0], cents[verts,1])))
        return reduce(lambda x,y: x*y, p)
        
    curve_dict = {pos_num:plot_hulls(pos_num) for pos_num in np.unique(props_clean.pos_num.values).tolist()}
    to_plot = hv.HoloMap(curve_dict,kdims='pos_num').redim(x={'range':(0,img_width)},y={'range':(0,img_height)})

else:
    ds_props = hv.Dataset(props_clean)
    to_plot = ds_props.to(hv.Scatter,'centx','centy',groupby=['pos_num']).overlay().redim(centx={'range':(0,img_width)},centy={'range':(0,img_height)})
    
    
to_plot

## plot filtered connected components and centroids

This plots the connected components pulled directly from your 'props_clean' pandas dataframe. This is the baseline truth of what is in your data. <br> <br>
It also overlays the centroids for the connected components. Missing cells at the bottom are expected (you filtered them out above). <br> <br>
If you are not keeping the correct amount of cells, you need to go back to [filter](#Analyze-and-filter-data).
If you are missing mother cells, you need to adjust either the mother_cell_offset or above_trench_quantile. If you have too few daughter cells adjust below_trench_quantile.

Once again, the colors are only indicative of connected components in a single image, they don't indicate lineages across time. i.e. it is expected that a mother cell will change color as it moves through lineages, don't panic! We haven't found lineages yet afterall, that will happen below. 

In [ ]:
lanes_analyzed = np.unique(props_clean.lane_num)
pos_analyzed = np.unique(props_clean.pos_num)
t_indcs_analyzed = np.unique(props_clean.t_frame)
conn_comp_results = PlotConnCompResults(lanes_analyzed, pos_analyzed, t_indcs_analyzed,props_clean)
conn_comp_results.plot()

## Sort properties into lineages based on positions

In [ ]:
start = timer()
props_sort = sort_cells_into_lineages(props_clean)
props_sort = determine_position_of_cell_in_trench(props_sort)
props_sort = create_linear_index(props_sort)
end = timer()   
print("time elapsed: %2.3f" % (end-start))

### Plot trenches to make sure they are correct

This is the same as the plots from above, but now the cells are colored by lineage, rather than connected component. You can tell whether two cells have been seperated by whether they have a centroid plotted on top of them, i.e. even if they look connected, if all the cells have centroids plotted on them, they have been detected. 

In [ ]:
lanes_analyzed = np.unique(props_sort.lane_num)
pos_analyzed = np.unique(props_sort.pos_num)
t_indcs_analyzed = np.unique(props_sort.t_frame)
conn_comp_results = PlotConnCompResults(lanes_analyzed, pos_analyzed, t_indcs_analyzed,props_sort)
conn_comp_results.plot()

Plot the convex hull of the centroids for a given lineage z-stacked through time for each position. The color should indicate the lineage. On top of that hull, is a point that gives the details for each lineage. <br> <br> 
Note: There seems to be a weird bug where the first time you go to a position, some hulls are white, just go forward a position and come back. <br>
Note: The initial zoom is currently a bit weird, you will need to zoom out. <br>
Also: You can zoom the axes independently by selecting the zoom tool and putting the cursor overtop of the axis.

In [ ]:
%%opts Scatter [title_format='hull of lineage centroids positions' width=int(img_width) height=200 invert_yaxis=True fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':10}] (size=6) {+axiswise}
%%opts Curve [width=int(img_width) height=200 invert_yaxis=True fontsize={'title':5, 'xlabel':10, 'ylabel':10, 'ticks':10}] {+axiswise}

n_trench_pos = len(props_sort.trench_inversion_mult.unique())
n_lineages_max = props_sort.lineage_idx.max()
plot_colors=hv.plotting.util.process_cmap('jet',ncolors=5)

def plot_lineage_hulls(pos_num):
    t_tips = [("centx", "@centx"),("centy", "@centy"),("area", "@area"),
              ("solidity", "@solidity"),("label","@label_orig"),("lineage","@linear_lineage_idx")]

    vdims = ["centx","centy","area","solidity","pos_num","lineage_idx","linear_lineage_idx"]
    hv_plot_list = []
    for trench_mult in np.unique(props_sort.trench_inversion_mult.values).tolist():
        to_select = np.all([props_sort.pos_num == pos_num, props_sort.trench_inversion_mult == trench_mult],axis=0)
        ps = props_sort[to_select]
        lin_idx = np.unique(ps.lineage_idx.values)
        p=[]
        centx=[]
        for lineage in lin_idx:
            to_select_lineage = ps.lineage_idx == lineage
            cents = ps.loc[to_select_lineage,['centx','centy']].values
            hull = ConvexHull(cents)
            verts = np.append(hull.vertices,hull.vertices[0])
            centx.append(np.mean(cents[verts,0]))
            p.append(hv.Curve((cents[verts,0], cents[verts,1])))

        p = [p[sb] for sb in np.argsort(centx)]
        ps_no_obj = ps.select_dtypes(exclude=np.object).copy()
        ps_no_obj['lineage_keys'] = ps_no_obj['linear_lineage_idx']
        ps_mean = ps_no_obj.groupby('lineage_keys').mean()
        ps_dataset = hv.Dataset(ps_mean,['centx','centy'],vdims=vdims)
        hover = HoverTool(tooltips=t_tips)
        hover.point_policy = 'snap_to_data'
        hv_cents = ps_dataset.to(hv.Scatter).opts(plot=dict(tools=[hover], color_index='lane_num'))
        p_hull = reduce(lambda x,y: x*y, p)
        hv_plot_list.append(hv_cents*p_hull)
        
    hv_plot = reduce(lambda x,y: x+y, hv_plot_list)
    return hv_plot.cols(1)

pos_nums = np.unique(props_sort.pos_num.values).tolist()
curve_dict = {pos_num:plot_lineage_hulls(pos_num) for pos_num in pos_nums}
to_plot = hv.HoloMap(curve_dict,kdims='pos_num').redim(x={'range':(0,img_width)})
to_plot.options({'Curve':dict(color=hv.Cycle(plot_colors)),'Scatter':dict(color='black')}).collate()

# Reorganize, clean and analyze lineages

## index by lineages, remove bad lineages, calculate division properties

This will help filter out bad lineages, i.e. those that are probably multiple lanes fused together and those that have too many nans

In [ ]:
mother_props = stack_mother_properties(props_sort)
# drop lineages with more more than max_frac_nan area measurements
max_frac_nan = 0.9

not_too_many_nulls = mother_props.area.notnull().sum() > mother_props.shape[0]*max_frac_nan

# remove lineages that contain two (or more) trenches grouped together during clustering
standard_trench_width = np.median(mother_props.centx.max() - mother_props.centx.min())
normal_width_lineages = mother_props.centx.max() - mother_props.centx.min() < 2*standard_trench_width

good_lineages = np.where(np.all([not_too_many_nulls,normal_width_lineages],axis=0))[0]

props_filtered = props_sort[props_sort['linear_lineage_idx'].isin(good_lineages)].copy()
props_filtered = cleanup_lengths(props_filtered,cutoff_scale=0,minimum_peak_height=12)
props_filtered = calculate_and_append_divisions(props_filtered)
props_filtered = calculate_and_append_divisions(props_filtered,peak_column='major_axis_length_corrected',suffix='_corrected')

In [ ]:
mom_props_filtered = props_filtered[props_filtered.cell_pos == 0]
dataset = hv.Dataset(mom_props_filtered.drop(['bbox','intensity_image'],axis=1))

q_dividing_cells = (mom_props_filtered[['is_dividing_corrected','linear_lineage_idx']].groupby('linear_lineage_idx').sum() > 0).values
dividing_cells = np.where(q_dividing_cells[:,0])[0].tolist()
dividing_mothers = mom_props_filtered[mom_props_filtered['linear_lineage_idx'].isin(dividing_cells)]
dividing_dataset = hv.Dataset(dividing_mothers.drop(['bbox','intensity_image'],axis=1))

## Plot traces and divisions

This shows the simple way in which your data can be easily plotted for various features, it is very minimal code for a pretty complex plot

In [ ]:
%%opts Curve [width=700 height=300 invert_yaxis=False fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':10}] {+axiswise}
%%opts Scatter (size=10,color='green')

hv_plot = dividing_dataset.to(hv.Curve,'t_frame','major_axis_length_corrected',groupby='linear_lineage_idx',dynamic=True).options(color='b')
hv_plot_un = dividing_dataset.to(hv.Curve,'t_frame','major_axis_length',groupby='linear_lineage_idx',dynamic=True).options(color='r')
new_dataset = dividing_dataset.iloc[dividing_dataset['is_dividing_corrected'].tolist(),:]

hv_scatter = new_dataset.to(hv.Scatter, 't_frame','major_axis_length_corrected',groupby='linear_lineage_idx',dynamic=True)
hv_scatter*hv_plot_un*hv_plot

A montage showing the mother cells and it's major axis length, as well as the found division times. The corrected major axis length is in blue and the original is in red, which will obviously only show up when you have a corrected timepoint

In [ ]:
%%opts Curve [width=1000 height=300 invert_yaxis=False fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':10}] {+axiswise}
%%opts Image [width=1000 height=300 invert_yaxis=False fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':10}] {+framewise}
%%opts Scatter (size=10)

lindcs = np.unique(props_filtered.linear_lineage_idx)
nan_frames = np.nan*np.ones((t_indcs_analyzed.shape[0]+1,))

def make_montage(lidx,t):
    ps = mom_props_filtered[mom_props_filtered.linear_lineage_idx == lidx]
    int_images = ps.intensity_image
    t_frames = ps.t_frame.tolist()
    t_frames_filled = nan_frames.copy()
    t_frames_filled[t_frames] = t_frames
    t_frames_filled = pd.Series(t_frames_filled).fillna(method='ffill')
    t_frames_filled = t_frames_filled.fillna(method='bfill').astype(np.int16)
    t_plot = t_frames_filled.iloc[t]
    
    c = 'b' if (t in t_frames) else 'r'
    t_final = np.where(t_frames == t)[0][0]+1
    img = make_cell_montage(np.array(int_images)[0:t_final],n_cols=50)
    h, w = img.shape[0], img.shape[1]
    
    hv_img = hv.Image(img,extents=(0,0,w,300),bounds=hv.BoundingBox(points=((0,0),(w,(1000/w)*h))))
    
    hv_sel = dataset.select(linear_lineage_idx=[lidx],cell_pos=0)
    hv_sel['major_axis_length']
    hv_curve = hv_sel.to(hv.Curve,'t_frame','major_axis_length',groupby='linear_lineage_idx').options(color='r').values()[0]
    hv_curve_cor = hv_sel.to(hv.Curve,'t_frame','major_axis_length_corrected',groupby='linear_lineage_idx').options(color='b').values()[0]

    hv_diff = hv_sel.to(hv.Curve,'t_frame','major_axis_length',groupby='linear_lineage_idx').values()[0]

    
    
    hv_scat = dataset.select(linear_lineage_idx=[lidx],cell_pos=0,t_frame=t_plot)
    hv_scat = hv_scat.to(hv.Scatter,'t_frame','major_axis_length',groupby=['linear_lineage_idx']).opts(style=dict(color=c)).values()[0]
    
    return (hv_img + hv_curve*hv_curve_cor*hv_scat).cols(1)

kdims = [hv.Dimension("lineage",values=lindcs),
         hv.Dimension("t_frame",values=t_indcs_analyzed)]
dmap = hv.DynamicMap(make_montage,kdims=kdims)
dmap

In [ ]:
save_dir = generate_save_directory(lane_num)
save_properties(props_all, os.path.join(save_dir, "props_all.pkl"))
save_properties(props_filtered, os.path.join(save_dir, "props_filtered.pkl"))